Wahlpflichtfach Künstliche Intelligenz II: Praktikum 

---

# 05 - Ensembles Learning mit Tensorflow

Nachdem wir uns angesehen haben, wie wir das Ergebnis durch die Optimierung der Hyperparameter verbessern versuchen wir das nun durch Ensemble learning.  

In [ ]:
import tensorflow as tf

In [2]:
%matplotlib inline

In [ ]:
print(tf.__version__)

## Erstellen eines kleinen klassifikationsdaten

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

iris = datasets.load_iris()
y = to_categorical(iris.target)
x = iris.data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

## Ensembles

Bei einem Ensemble handelt es sich um einen Zusammenschluss mehrerer KI-Modelle. Hierbei gibt es zum Beispiel die Möglichkeit, die den durchschnitt oder das Maximum der unterschiedlichen Modelle zu nehmen. Diese können dabei auch beliebig tief geschachtelt werden.

In [ ]:
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy, Recall, Precision

model_1 = Sequential(name="classification_mlp1")
model_1.add(Dense(8, activation='relu', input_dim=4))
model_1.add(Dense(8, activation='relu'))
model_1.add(Dense(8, activation='relu'))
model_1.add(Dense(3, activation='softmax'))

model_1.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy(), Recall(), Precision()]
)

model_1.fit(
    x=x_train, 
    y=y_train,
    validation_split=0.2,
    epochs=5,
)
loss, acc, rec, prec = model_1.evaluate(x_test, y_test)
tf.keras.backend.clear_session()

In [ ]:
model_2 = Sequential(name="classification_mlp2")
model_2.add(Dense(16, activation='relu', input_dim=4))
model_2.add(Dense(8, activation='relu'))
model_2.add(Dense(4, activation='relu'))
model_2.add(Dense(3, activation='softmax'))

model_2.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy(), Recall(), Precision()]
)

model_2.fit(
    x=x_train, 
    y=y_train,
    validation_split=0.2,
    epochs=5,
)
loss, acc, rec, prec = model_2.evaluate(x_test, y_test)
tf.keras.backend.clear_session()

**Average Ensemble**

In [ ]:
from keras.layers import Input, Average, Maximum
from keras.models import Model
from keras.utils import plot_model

models = [model_1, model_2]

model_input = Input(shape=(4,))
single_mlp_outputs = [model(model_input) for model in models]
ensemble_output = Average()([
    model(model_input)
    for model in models
])
ensemble_model_avg = Model(inputs=model_input, outputs=ensemble_output)
plot_model(ensemble_model_avg, show_shapes=True, show_dtype=False, show_layer_names=True,
           show_layer_activations=True)

In [ ]:
ensemble_model_avg.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy(), Recall(), Precision()]
)
loss, acc, rec, prec = ensemble_model_avg.evaluate(x_test, y_test)

**Maximum Ensemble**

In [ ]:
model_input = Input(shape=(4,))
single_mlp_outputs = [model(model_input) for model in models]
ensemble_output = Maximum()([
    model(model_input)
    for model in models
])
ensemble_model_max = Model(inputs=model_input, outputs=ensemble_output)
plot_model(ensemble_model_max, show_shapes=True, show_dtype=False, show_layer_names=True,
           show_layer_activations=True)

In [ ]:
ensemble_model_max.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy(), Recall(), Precision()]
)
loss, acc, rec, prec = ensemble_model_max.evaluate(x_test, y_test)

**Majority Voting Ensemble**

In [11]:
from keras.src.layers.merging.base_merge import _Merge

class MajorityVoting(_Merge):
    def _clean(self, t: tf.Tensor):
        top, _ = tf.math.top_k(t, 1)
        return tf.cast(tf.equal(t, top), dtype=tf.float32)

    def _merge_function(self, inputs: list[tf.Tensor]):
        output = self._clean(inputs[0])
        for i in range(1, len(inputs)):
            output = tf.add(output, self._clean(inputs[i]))
        return tf.divide(output, len(inputs))

In [ ]:
model_input = Input(shape=(4,))
single_mlp_outputs = [model(model_input) for model in models]
ensemble_output = MajorityVoting()([
    model(model_input)
    for model in models
])
ensemble_model_voting = Model(inputs=model_input, outputs=ensemble_output)
plot_model(ensemble_model_max, show_shapes=True, show_dtype=False, show_layer_names=True,
           show_layer_activations=True)

In [ ]:
ensemble_model_voting.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy(), Recall(), Precision()]
)
loss, acc, rec, prec = ensemble_model_voting.evaluate(x_test, y_test)

---

Wahlpflichtach Künstliche Intelligenz II: Praktikum 